In [3]:
import pandas as pd
import numpy as np
from itertools import islice
import os
import pdb

def genotypeOfHaplotypes(genotype_info):
    
    FirstPart = genotype_info.split(":")[0];
    
    if not ("|" in FirstPart):
        return "22";
    
    FirstPartSplit = FirstPart.split("|");
    
    genotype_hap1 = 0 if FirstPartSplit[0] == "0" else 1;
    genotype_hap2 = 0 if FirstPartSplit[1] == "0" else 1;
    return (str(genotype_hap1) + str(genotype_hap2));




maximum_number_varinats = 16
# where individuals id start in vcf file
individual_starting_column = 8 
for variationCount in range(1,maximum_number_varinats):



    lookup_table= pd.read_table("lookupTable_variantNo_"+str(variationCount)+".txt", sep=" ");
    genotype_info= pd.read_table("data\vcf_sample.txt", sep = "\t");
# sort lookup table 
    lookup_table = lookup_table.sort_values(by=['variant_id_1'])    

    lookup_table = lookup_table.reset_index(drop= True)  #make indexes start at 0

    
    prediction_matrix_haplotype1= pd.DataFrame()
    prediction_matrix_haplotype2= pd.DataFrame()

    hap1_individual_genotype_lst = []
    hap2_individual_genotype_lst = []
    temp = pd.DataFrame();
    
    pred_index =1 
    individuals = genotype_info.columns.tolist()[individual_starting_column:]
    individual_info_hap1 = pd.DataFrame(columns = individuals)
    individual_info_hap2 = pd.DataFrame(columns = individuals)

    genotype_info_hap1 = pd.DataFrame(columns = individuals)
    genotype_info_hap2 = pd.DataFrame(columns = individuals)

    for index, row in lookup_table.iterrows():
    
        hap1_individual_genotype_lst.clear()
        hap2_individual_genotype_lst.clear()
        columns = pd.Series()
        columns = columns.append(pd.Series({'gene_id':row['gene_id']}))
        for i in range(1,variationCount+1):
            var= 'variant_id_' + str(i)
            columns = columns.append(pd.Series({var:row[var]}))
            temp = columns.to_frame().T 
            key = row['gene_id']+ "_"+row[var]
            print (key)
        
# find the gene in genotype file
        
            genotype_index = list(np.where(genotype_info["ID"] == row[var]))[0]
#In case there is gene_id information in the genotype file
# genotype_index = list(np.where(genotype_info["gene_id_ID"] == key))[0]
            if (len(genotype_index)==0):     # chrX won't be found
                continue
#print (genotype_index[0]);
            hap_ind = 0
        
            for ind in individuals:
            
# make genotypes            
                genotype_both_haps= genotypeOfHaplotypes(genotype_info.loc[genotype_index[0],ind])
            
                if (i>1):
                    hap1_individual_genotype_lst.insert(hap_ind, hap1_individual_genotype_lst[hap_ind]+str(genotype_both_haps[0]) + ", ")
                    del hap1_individual_genotype_lst[hap_ind + 1]
                    hap2_individual_genotype_lst.insert(hap_ind, hap2_individual_genotype_lst[hap_ind]+str(genotype_both_haps[1]) + ", ")
                    del hap2_individual_genotype_lst[hap_ind + 1]
                else:
                    hap1_individual_genotype_lst.insert(hap_ind, str(genotype_both_haps[0]) + ", ")
                    hap2_individual_genotype_lst.insert(hap_ind, str(genotype_both_haps[1]) + ", ")
                hap_ind = hap_ind+1
   
    
    
    
    
    
    
        prediction_matrix_haplotype1= prediction_matrix_haplotype1.append(temp,ignore_index=True)
        prediction_matrix_haplotype2= prediction_matrix_haplotype2.append(temp,ignore_index=True)
    
        i = 0
        for genotype in hap1_individual_genotype_lst:
        
            genotype_info_hap1.at[index, individuals[i]] =  genotype[:-2]
            if ("2" in genotype[:-2] ):
# non-phased data
                individual_info_hap1.at[index, individuals[i]] = "NA"
        
            else:
# find the genotype in lookup table
                individual_info_hap1.at[index, individuals[i]] =  row[genotype[:-2]]
            i = i+1
    
        i=0;    
        for genotype in hap2_individual_genotype_lst:
        
            genotype_info_hap2.at[index, individuals[i]] =  genotype[:-2]
            if ("2" in genotype[:-2] ):
                individual_info_hap2.at[index, individuals[i]] = "NA"
        
            else:
        
                individual_info_hap2.at[index, individuals[i]] =  row[genotype[:-2]]
            i = i+1
        
        
        
    output_hap1 = pd.concat([prediction_matrix_haplotype1, individual_info_hap1],axis = 1, sort = False)
    output_hap2 = pd.concat([prediction_matrix_haplotype2, individual_info_hap2],axis = 1, sort = False)

    output_genotype_hap1 = pd.concat([prediction_matrix_haplotype1, genotype_info_hap1],axis = 1, sort = False)
    output_genotype_hap2 = pd.concat([prediction_matrix_haplotype2, genotype_info_hap2],axis = 1, sort = False)

#outputs
    output_hap1.to_csv('ASE_prediction_hap1_Variant_Num_'+str(variationCount)+".csv")
    output_hap2.to_csv('ASE_prediction_hap2_Variant_Num_'+str(variationCount)+".csv") 
    output_genotype_hap1.to_csv('genotype_hap1_Variant_Num_'+str(variationCount)+".csv")
    output_genotype_hap2.to_csv('genotype_hap2_Variant_Num_'+str(variationCount)+".csv")    




/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: FutureWarning: read_table is deprecated, use read_csv instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: FutureWarning: read_table is deprecated, use read_csv instead.


ENSG00000204172_chr10_47527652_T_TTG_b38
ENSG00000204172_chr10_46805416_G_A_b38
ENSG00000204172_chr10_46767155_G_A_b38
ENSG00000204172_chr10_47737681_G_GT_b38
ENSG00000204172_chr10_48032196_A_G_b38
ENSG00000204172_chr10_47357676_C_T_b38
ENSG00000204172_chr10_47557412_C_A_b38
ENSG00000223482_chr10_87312886_CCAA_C_b38
ENSG00000223482_chr10_88142437_A_G_b38
ENSG00000223482_chr10_87643839_A_T_b38
ENSG00000223482_chr10_87263615_C_T_b38
ENSG00000223482_chr10_86984351_CATT_C_b38
ENSG00000223482_chr10_86958394_C_A_b38
ENSG00000223482_chr10_87214060_C_T_b38
ENSG00000111788_chr12_9280307_G_A_b38
ENSG00000111788_chr12_9318173_A_G_b38
ENSG00000111788_chr12_9311178_G_T_b38
ENSG00000111788_chr12_9296350_G_A_b38
ENSG00000111788_chr12_9107684_T_C_b38
ENSG00000111788_chr12_9313096_T_C_b38
ENSG00000111788_chr12_9305257_C_G_b38
ENSG00000187630_chr14_23970644_C_T_b38
ENSG00000187630_chr14_23959742_C_T_b38
ENSG00000187630_chr14_23718098_G_GATTTTGACCCAAAGATATAATCAGGTGTGGAAGCA_b38
ENSG00000187630_chr14_23964

ENSG00000228960_chr7_144172128_T_A_b38
ENSG00000228960_chr7_144276745_C_A_b38
ENSG00000228960_chr7_144316026_G_T_b38
ENSG00000228960_chr7_144445562_C_CG_b38
ENSG00000228960_chr7_144171231_T_C_b38
ENSG00000228960_chr7_144363193_A_G_b38
ENSG00000228960_chr7_144256440_C_T_b38
ENSG00000002933_chr7_150210827_G_C_b38
ENSG00000002933_chr7_149823661_G_A_b38
ENSG00000002933_chr7_150804228_T_C_b38
ENSG00000002933_chr7_150800408_C_A_b38
ENSG00000002933_chr7_150770290_G_T_b38
ENSG00000002933_chr7_150800957_A_G_b38
ENSG00000002933_chr7_150826699_T_G_b38
ENSG00000196204_chr7_4078201_C_T_b38
ENSG00000196204_chr7_5002126_A_T_b38
ENSG00000196204_chr7_4831481_C_T_b38
ENSG00000196204_chr7_4981121_G_A_b38
ENSG00000196204_chr7_5322494_T_C_b38
ENSG00000196204_chr7_5062865_A_C_b38
ENSG00000196204_chr7_4979851_G_C_b38
ENSG00000242611_chr7_94384_GA_G_b38
ENSG00000242611_chr7_78927_G_A_b38
ENSG00000242611_chr7_38119_C_T_b38
ENSG00000242611_chr7_68482_G_T_b38
ENSG00000242611_chr7_87129_C_T_b38
ENSG00000242611_ch

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: FutureWarning: read_table is deprecated, use read_csv instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: FutureWarning: read_table is deprecated, use read_csv instead.


ENSG00000230869_chr10_45803982_G_A_b38
ENSG00000230869_chr10_45838876_G_A_b38
ENSG00000230869_chr10_45701058_A_G_b38
ENSG00000230869_chr10_46647672_C_T_b38
ENSG00000230869_chr10_45616502_A_G_b38
ENSG00000230869_chr10_45701058_A_G_b38
ENSG00000230869_chr10_45837634_A_C_b38
ENSG00000230869_chr10_45348828_C_A_b38
ENSG00000204177_chr10_46994625_C_T_b38
ENSG00000204177_chr10_47647724_C_T_b38
ENSG00000204177_chr10_46869339_T_C_b38
ENSG00000204177_chr10_46874923_G_T_b38
ENSG00000204177_chr10_46804736_G_T_b38
ENSG00000204177_chr10_46543875_C_T_b38
ENSG00000204177_chr10_46746737_T_G_b38
ENSG00000204177_chr10_46898848_C_T_b38
ENSG00000255769_chr15_82139965_T_C_b38
ENSG00000255769_chr15_82498293_C_A_b38
ENSG00000255769_chr15_82411134_C_T_b38
ENSG00000255769_chr15_82404864_T_C_b38
ENSG00000255769_chr15_82331957_C_G_b38
ENSG00000255769_chr15_82395506_A_G_b38
ENSG00000255769_chr15_83399149_T_TA_b38
ENSG00000255769_chr15_82201074_C_G_b38
ENSG00000183426_chr16_15036083_C_T_b38
ENSG00000183426_chr16_14